In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4225157188458270974, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9425665546910455468
 physical_device_desc: "device: XLA_CPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2304114688
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14481774252847935662
 physical_device_desc: "device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4357901821301314906
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
def load_all_cifar_data():
    folder_path = 'cifar-10-batches-py'
    data = []
    labels = []
    for i in range(1,6):
        batch = unpickle(f'{folder_path}/data_batch_{i}')
        data.append(batch[b'data'])
        labels.append(batch[b'labels'])
    
    return np.concatenate(tuple(data), axis=0), np.concatenate(tuple(labels), axis=0)

In [5]:
data, labels = load_all_cifar_data()

In [6]:
a = np.zeros((50000,10))
a[np.arange(50000), labels] = 1

labels = a
print(data.shape)
print(labels.shape)

(50000, 3072)
(50000, 10)


In [7]:
data = np.divide(data, 255)

# Simple NN

In [ ]:
model = tf.keras.Sequential()
from tensorflow.keras.layers import Dense
model.add(Dense(units=1024, activation='relu', input_dim=3072))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(data, labels, validation_split=0.20, epochs=10, batch_size=64)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# model.save('simple-nn-100-epochs.h5')

# CNN

In [8]:
def convert_data_for_cnn(data):
    num_entries_in_channel = int(data.shape[1] / 3)
    data_4d = np.zeros((50000,3,32,32))
    for i in range(50000):
        x = data[i]
        r_1d = x[0:num_entries_in_channel]
        g_1d = x[num_entries_in_channel:2*num_entries_in_channel]
        b_1d = x[num_entries_in_channel*2:]

        r_2d = np.reshape(r_1d, (32,32))
        g_2d = np.reshape(g_1d, (32,32))
        b_2d = np.reshape(b_1d, (32,32))
        
        image_3d = np.zeros((3,32,32))
        image_3d[0] = r_2d
        image_3d[1] = g_2d
        image_3d[2] = b_2d
                
        data_4d[i] = image_3d
    print(data_4d.shape)
    data_4d = np.swapaxes(data_4d,1,3)
    data_4d = np.swapaxes(data_4d, 1,2)
    
    mean = np.mean(data_4d,axis=(0,1,2,3))
    std = np.std(data_4d,axis=(0,1,2,3))
    data_4d = (data_4d-mean)/(std+1e-7)
    return data_4d

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# initiate RMSprop optimizer
opt = RMSprop(learning_rate=0.001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(convert_data_for_cnn(data), labels, validation_split=0.2, epochs=15, batch_size=64, shuffle=True)

In [ ]:
# model.save('simple-cnn-model-100-epoch.h5')

# CNN WITH SEPERABLE CONVOLUTIONS

In [17]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, DepthwiseConv2D, BatchNormalization
from tensorflow.keras.models import Sequential

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16, kernel_size=(1,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, kernel_size=(1,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))

In [18]:
from tensorflow.keras.optimizers import RMSprop

# initiate RMSprop optimizer
opt = RMSprop(learning_rate=0.001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_27 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_conv2d_3 (Depthwis (None, 16, 16, 32)        320       
_________________________________________________________________
batch_normalization_16 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
activation_28 (Activation)   (None, 16, 16, 32)       

In [19]:
history = model.fit(convert_data_for_cnn(data), labels, validation_split=0.2, epochs=30, batch_size=64, shuffle=True)

(50000, 3, 32, 32)
Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 14s 344us/sample - loss: 1.7637 - acc: 0.3625 - val_loss: 1.5887 - val_acc: 0.4235
Epoch 2/30
40000/40000 [==============================] - 13s 334us/sample - loss: 1.3998 - acc: 0.4966 - val_loss: 1.3393 - val_acc: 0.5173
Epoch 3/30
40000/40000 [==============================] - 13s 325us/sample - loss: 1.2745 - acc: 0.5442 - val_loss: 1.2637 - val_acc: 0.5493
Epoch 4/30
40000/40000 [==============================] - 13s 313us/sample - loss: 1.1873 - acc: 0.5778 - val_loss: 1.2208 - val_acc: 0.5712
Epoch 5/30
40000/40000 [==============================] - 12s 310us/sample - loss: 1.1238 - acc: 0.6008 - val_loss: 1.1489 - val_acc: 0.5947
Epoch 6/30
40000/40000 [==============================] - 12s 311us/sample - loss: 1.0801 - acc: 0.6165 - val_loss: 1.1345 - val_acc: 0.6059
Epoch 7/30
40000/40000 [==============================] - 12s 309us/sample - loss: 1.

# CNN 64x3

In [11]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

model = Sequential()

weight_decay = 0.0001
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))

In [12]:
from tensorflow.keras.optimizers import Adam

# initiate RMSprop optimizer
opt = Adam(learning_rate=0.001)

# Let's train the model using Adam
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 32)       

In [13]:
history = model.fit(convert_data_for_cnn(data), labels, validation_split=0.2, epochs=30, batch_size=64, shuffle=True)

(50000, 3, 32, 32)
Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 13s 337us/sample - loss: 1.8138 - acc: 0.4107 - val_loss: 1.3153 - val_acc: 0.5414
Epoch 2/30
40000/40000 [==============================] - 12s 312us/sample - loss: 1.2104 - acc: 0.5878 - val_loss: 1.0558 - val_acc: 0.6450
Epoch 3/30
40000/40000 [==============================] - 13s 321us/sample - loss: 0.9790 - acc: 0.6728 - val_loss: 0.8977 - val_acc: 0.7020
Epoch 4/30
40000/40000 [==============================] - 13s 319us/sample - loss: 0.8582 - acc: 0.7167 - val_loss: 0.7853 - val_acc: 0.7456
Epoch 5/30
40000/40000 [==============================] - 13s 330us/sample - loss: 0.7836 - acc: 0.7452 - val_loss: 0.8113 - val_acc: 0.7421
Epoch 6/30
40000/40000 [==============================] - 13s 336us/sample - loss: 0.7277 - acc: 0.7697 - val_loss: 0.7667 - val_acc: 0.7577
Epoch 7/30
40000/40000 [==============================] - 13s 325us/sample - loss: 0.